Это 4 лаба по питону.

In [1]:
import cv2
import pandas as pd

1. Формируем DataFrame:

In [16]:
df = pd.read_csv('dataset.csv', sep = ' ')

,Absolute_way,Class
0,E:\dataset/polarbears/0000.jpg,polarbears
1,E:\dataset/polarbears/0001.jpg,polarbears
2,E:\dataset/polarbears/0002.jpg,polarbears
3,E:\dataset/polarbears/0003.jpg,polarbears
4,E:\dataset/polarbears/0004.jpg,polarbears
5,E:\dataset/polarbears/0005.jpg,polarbears
6,E:\dataset/polarbears/0006.jpg,polarbears
7,E:\dataset/polarbears/0007.jpg,polarbears
8,E:\dataset/polarbears/0008.jpg,polarbears
